In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 层和块

首先，我们回顾一下多层感知机

In [3]:
torch::nn::Sequential net(torch::nn::Linear(20, 256),
                          torch::nn::ReLU(),
                          torch::nn::Linear(256, 10)
                         );
/* ***
 * 当然也可以给每个layer加个名字
 */
// torch::nn::Sequential net1({{"fc1", torch::nn::Linear(20, 256)},
//                           {"relu1", torch::nn::ReLU()},
//                           {"fc2", torch::nn::Linear(256, 10)}}
//                          );


auto X = torch::rand({2, 20});

$\uparrow\uparrow\uparrow\uparrow\uparrow$   nn.Sequential定义了一种特殊的Module

## 自定义块

In [4]:
/*design a net*/
struct MLP : torch::nn::Module {
  MLP() {
    // Construct and register two Linear submodules.
    hidden = register_module("hidden", torch::nn::Linear(20, 256));
    output = register_module("output", torch::nn::Linear(256, 10));
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(torch::Tensor x) {
    // Use one of many tensor manipulation functions.
    x = hidden->forward(x);
    x = torch::relu(x);
    x = output->forward(x);
    return x;
  }

  // Use one of many "standard library" modules.
  torch::nn::Linear hidden{nullptr}, output{nullptr};
};

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [5]:
// Create a new Net.
auto net = std::make_shared<MLP>();
printT(net->forward(X));

net->forward(X) = 
 0.0050 -0.0098 -0.1058  0.1871  0.1308 -0.2072  0.1157  0.0479  0.0384 -0.0533
 0.0095  0.0497 -0.2710  0.3096  0.2933 -0.1786  0.1169  0.1132  0.0158 -0.0726
[ CPUFloatType{2,10} ]
<<--->>



## 顺序块

原教程此部分功能类似于torch::nn::Sequential，所以不再赘述；
可以参考源码中关于Sequential类的实现：

```

 /// A `ModuleHolder` subclass for `SequentialImpl`.
/// See the documentation for `SequentialImpl` class to learn what methods it
/// provides, or the documentation for `ModuleHolder` to learn about PyTorch's
/// module storage semantics.                                                                                                                                                        
class Sequential : public torch::nn::ModuleHolder<SequentialImpl> {
public:
  using torch::nn::ModuleHolder<SequentialImpl>::ModuleHolder;

  Sequential() : ModuleHolder() {}

  /// Constructs the `Sequential` from a braced-init-list of named `AnyModule`s.
  /// It enables the following use case:
  /// `Sequential sequential({{"m1", M(1)}, {"m2", M(2)}})`
  Sequential(std::initializer_list<NamedAnyModule> named_modules) : ModuleHolder(std::make_shared<SequentialImpl>(std::move(named_modules))) {}
 
};

```

在正向传播函数中执行代码

In [6]:
/*design a net*/
struct FixedHiddenMLP : torch::nn::Module {
    FixedHiddenMLP() {
        rand_weight = torch::rand({20, 20}, torch::requires_grad(false));
        linear = register_module("linear", torch::nn::Linear(20, 20));
    }

    // Implement the FixedHiddenMLP's algorithm.
    torch::Tensor forward(torch::Tensor x) {
        // Use one of many tensor manipulation functions.
        x = linear->forward(x);
        x = torch::relu(torch::mm(x, rand_weight)+1);
        x = linear->forward(x);
        while (x.abs().sum().item<float>() > 1) {
            x /= 2;
        }
        
        return x.sum();
    }

  // Use one of many "standard library" modules.
  torch::Tensor rand_weight;
  torch::nn::Linear linear{nullptr};
};

In [7]:
// Create a new Net.
auto net3 = std::make_shared<FixedHiddenMLP>();
printT(net3->forward(X));

net3->forward(X) = 
-0.0140377
[ CPUFloatType{} ]
<<--->>



## 混合搭配各种组合块的方法

In [8]:
/*design a net*/
struct NestMLP : torch::nn::Module {
    NestMLP() {
        net = torch::nn::Sequential(torch::nn::Linear(20, 64),
                                    torch::nn::ReLU(),
                                    torch::nn::Linear(64, 32),
                                    torch::nn::ReLU());
        linear = torch::nn::Linear(32, 16);
    }

    // Implement the Net's algorithm.
    torch::Tensor forward(torch::Tensor x) {
        x = linear->forward(net->forward(x));
        return x;
    }

    // Use one of many "standard library" modules.
    torch::nn::Sequential net{nullptr};
    torch::nn::Linear linear{nullptr};
};

In [9]:
auto chimera = torch::nn::Sequential(NestMLP(), 
                                torch::nn::Linear(16, 20), 
                                FixedHiddenMLP());
printT(chimera->forward(X));

chimera->forward(X) = 
-0.113716
[ CPUFloatType{} ]
<<--->>

